# 如何将工具输出传递给聊天模型

```{=mdx}
:::info 预备知识
本指南假定您熟悉以下概念：

- [LangChain 工具](/docs/concepts/tools)
- [工具调用](/docs/concepts/tool_calling)
- [使用聊天模型进行工具调用](/docs/how_to/tool_calling)
- [定义自定义工具](/docs/how_to/custom_tools/)

:::
```

某些模型具备[**工具调用**](/docs/concepts/tool_calling)的能力，可以生成符合特定用户提供的模式的参数。本指南将演示如何使用这些工具调用来实际调用函数，并正确地将结果传递回模型。

![](../../static/img/tool_invocation.png)

![](../../static/img/tool_results.png)

首先，我们定义我们的工具和模型：

In [1]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";

const addTool = tool(async ({ a, b }) => {
  return a + b;
}, {
  name: "add",
  schema: z.object({
    a: z.number(),
    b: z.number(),
  }),
  description: "Adds a and b.",
});

const multiplyTool = tool(async ({ a, b }) => {
  return a * b;
}, {
  name: "multiply",
  schema: z.object({
    a: z.number(),
    b: z.number(),
  }),
  description: "Multiplies a and b.",
});

const tools = [addTool, multiplyTool];

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

现在，让我们让模型调用一个工具。我们会将它添加到一个消息列表中，该列表将被视为对话历史记录：

In [2]:
import { HumanMessage } from "@langchain/core/messages";

const llmWithTools = llm.bindTools(tools);

const messages = [
  new HumanMessage("What is 3 * 12? Also, what is 11 + 49?"),
];

const aiMessage = await llmWithTools.invoke(messages);

console.log(aiMessage);

messages.push(aiMessage);

AIMessage {
  "id": "chatcmpl-9p1NbC7sfZP0FE0bNfFiVYbPuWivg",
  "content": "",
  "additional_kwargs": {
    "tool_calls": [
      {
        "id": "call_RbUuLMYf3vgcdSQ8bhy1D5Ty",
        "type": "function",
        "function": "[Object]"
      },
      {
        "id": "call_Bzz1qgQjTlQIHMcEaDAdoH8X",
        "type": "function",
        "function": "[Object]"
      }
    ]
  },
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 50,
      "promptTokens": 87,
      "totalTokens": 137
    },
    "finish_reason": "tool_calls",
    "system_fingerprint": "fp_400f27fa1f"
  },
  "tool_calls": [
    {
      "name": "multiply",
      "args": {
        "a": 3,
        "b": 12
      },
      "type": "tool_call",
      "id": "call_RbUuLMYf3vgcdSQ8bhy1D5Ty"
    },
    {
      "name": "add",
      "args": {
        "a": 11,
        "b": 49
      },
      "type": "tool_call",
      "id": "call_Bzz1qgQjTlQIHMcEaDAdoH8X"
    }
  ],
  "invalid_tool_calls": [],
  "usage_metadata": {
   

接下来，让我们使用模型填充的参数调用工具函数！

方便的是，如果我们使用 `ToolCall` 调用一个 LangChain `Tool`，我们将自动收到一个 `ToolMessage`，该消息可以反馈给模型：

```{=mdx}
:::caution 兼容性

此功能需要 `@langchain/core>=0.2.16`。请参阅[升级指南](/docs/how_to/installation/#installing-integration-packages)。

如果您使用的是较早版本的 `@langchain/core`，则需要使用工具调用中的字段手动构造一个 `ToolMessage`。

:::
```

In [3]:
const toolsByName = {
  add: addTool,
  multiply: multiplyTool,
}

for (const toolCall of aiMessage.tool_calls) {
  const selectedTool = toolsByName[toolCall.name];
  const toolMessage = await selectedTool.invoke(toolCall);
  messages.push(toolMessage);
}

console.log(messages);

[
  HumanMessage {
    "content": "What is 3 * 12? Also, what is 11 + 49?",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "chatcmpl-9p1NbC7sfZP0FE0bNfFiVYbPuWivg",
    "content": "",
    "additional_kwargs": {
      "tool_calls": [
        {
          "id": "call_RbUuLMYf3vgcdSQ8bhy1D5Ty",
          "type": "function",
          "function": "[Object]"
        },
        {
          "id": "call_Bzz1qgQjTlQIHMcEaDAdoH8X",
          "type": "function",
          "function": "[Object]"
        }
      ]
    },
    "response_metadata": {
      "tokenUsage": {
        "completionTokens": 50,
        "promptTokens": 87,
        "totalTokens": 137
      },
      "finish_reason": "tool_calls",
      "system_fingerprint": "fp_400f27fa1f"
    },
    "tool_calls": [
      {
        "name": "multiply",
        "args": {
          "a": 3,
          "b": 12
        },
        "type": "tool_call",
        "id": "call_RbUuLMYf3vgcdSQ8bhy1D5Ty"
      },
      {
  

最后，我们将使用工具结果调用模型。模型将利用这些信息生成对我们原始查询的最终答案：

In [4]:
await llmWithTools.invoke(messages);

AIMessage {
  "id": "chatcmpl-9p1NttGpWjx1cQoVIDlMhumYq12Pe",
  "content": "3 * 12 is 36, and 11 + 49 is 60.",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 19,
      "promptTokens": 153,
      "totalTokens": 172
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_18cc0f1fa0"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 153,
    "output_tokens": 19,
    "total_tokens": 172
  }
}


请注意，每个 `ToolMessage` 必须包含一个与模型生成的原始工具调用中的 `id` 相匹配的 `tool_call_id`。这有助于模型将工具响应与工具调用进行匹配。

像 [LangGraph](https://langchain-ai.github.io/langgraphjs/tutorials/introduction/) 中的工具调用代理一样，使用这种基本流程来回答查询和解决问题。

## 相关内容

现在你已经了解了如何将工具调用传递回模型。

接下来你可能对以下指南感兴趣：

- [LangGraph 快速入门](https://langchain-ai.github.io/langgraphjs/tutorials/introduction/)
- 使用工具进行[少样本提示](/docs/how_to/tools_few_shot/)
- 流式传输[工具调用](/docs/how_to/tool_streaming/)
- 将[运行时值传递给工具](/docs/how_to/tool_runtime)
- 从模型获取[结构化输出](/docs/how_to/structured_output/)
